<a href="https://colab.research.google.com/github/beaten-by-the-market/dart_disclosure/blob/main/listing_special.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 기술성장 특례로 상장한 기업들은?
투자를 할 때 우리는 보통 기업의 가치를 평가하여 투자 대상을 선정합니다. 이 과정에서 재무정보는 가장 중요한 기준으로 작용합니다. <br>하지만 코스닥 시장에서 '기술성장기업'으로 특례상장된 기업들은 조금 다른 모습을 보입니다. 이들은 재무실적이 좋지 않더라도 기술력과 성장성을 인정받아 상장된 경우가 많습니다. 따라서 전통적인 재무분석 방식으로는 이 기업들의 매력을 제대로 평가하기 어려울 수 있습니다.<br><br>
이런 이유로 투자자 입장에서는 기술성장기업들을 별도의 투자 유니버스로 관리하고 싶어질 것입니다. 만약 기존의 재무기준 필터만을 사용한다면, 기술적으로 유망한 기업들을 놓칠 가능성이 높기 때문입니다.<br><br>
따라서 이들 기업에 대해서는 다른 평가기준이 필요합니다. 기술평가 특례와 성장성 특례와 같은 제도를 통해 상장된 기업들은 전문평가기관에서 기술력과 시장성을 인정받은 만큼, 투자자는 이를 중심으로 새로운 분석 프레임워크를 만들어야 합니다.<br><br>
분석을 논하기 전에... 일단 **"기술성장 특례로 상장한 기업은 어디일까?"**{: style="color: #4682B4;"}라는 질문으로 시작해보겠습니다. 언뜻 간단한 궁금증 같아 보이지만... 매우 기나긴 여정을 거쳐야 합니다.

## 파이썬 세팅하기

In [1]:
#-----------------------------------------------------------
#필요한 패키지 import
#-----------------------------------------------------------
import pandas as pd
import requests
from time import sleep
from tqdm import tqdm
from datetime import datetime, timedelta
from io import BytesIO
import numpy as np
from tqdm.notebook import tqdm
import warnings
from bs4 import BeautifulSoup
import re

# 시각화 관련
!pip install adjustText
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from adjustText import adjust_text

# 한글 폰트 설정 (Google Colab 환경에서)
!apt-get update -qq
!apt-get install fonts-nanum* -qq

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=10)

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package fonts-nanum.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-3_all.deb ...
Unpacking fonts-nanum-coding (2.5-3) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-7_all.deb ...
Unpacking fonts-nanum-eco (1.000-7) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20200506-1_all.deb ...
Unpacking fonts-nanum-extra (20200506-1) ...
Setting up fonts-nanum-extra (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Setting up fo

## 정보데이터시스템에서 상장트랙별 회사 불러오기
특례상장기업 목록 찾기.. 홈페이지 하나에 다 나와있지 않을까? 라는 가벼운 마음으로 시작해봅니다. 이전에도 봤었던 한국거래소의 [정보데이터시스템(링크)](http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0202)에 있을 것 같아서 한번 가보겠습니다.<br><br>
![datakrx]({{site.url}}/assets/images/2025-03-01-listing/datakrx.png)<br><br>
바로 눈에 띄진 않지만 그래도 'IPO' 관련 내용이니깐 관련 메뉴를 클릭하다보면 **"[20043] 공모가 대비 주가수익률"**{: style="color: #4682B4;"}이란 화면이 나옵니다. 쉽게 찾았네요!<br><br>
![datakrx2]({{site.url}}/assets/images/2025-03-01-listing/datakrx2.png)<br><br>
...가 아니라 저 화면은 **최근 5년 상장법인까지만**{: style="color: #4682B4;"} 보여줍니다. 기술성장기업에 대한 특례기간은 보통 5년 정도입니다. 예를들면, 관리종목 지정사유가 발생해도 봐주는 기간이 5년 정도입니다. 이런 점을 고려하면 합리적인 수준의 표출인 것 같기도 합니다만... **혁신기술이나 성장성을 인정받아 상장한 회사 전체를 별도 투자유니버스로 구성하여 확인하고싶다는 궁금증에는 턱없이 부족한 정보입니다. **{: style="color: #4682B4;"}
<br><br>
그리고 하나더. **저 화면에는 종목코드가 없습니다.**{: style="color: #4682B4;"} 엑셀로 다운받아 보아도 없습니다. 종목코드가 없이는 다른 데이터소스와 연계하여 분석을 하기가 힘듭니다. 지난번 사례에서 보았듯, KIND에서의 회사명과 정보데이터시스템에서의 회사명이 다른 경우가 있습니다.
> 회사명으로는 식별되지 않는 경우가 많기 때문에 반드시 종목코드가 필요합니다.


### 상장트랙별 회사를 불러왔는데...종목코드가 없다ㅜㅜ

In [2]:
#-----------------------------------------------------------
#정보데이터시스템에서 상장트랙 불러오는 과정
#-----------------------------------------------------------

#화면번호 [20043] 공모가 대비 주가수익률
#거래소 홈페이지에서 불러오기
gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp = {
    "locale": "ko_KR",
    "tboxisuCd_finder_stkisu4_3": "전체",
    "isuCd": "ALL",
    "isuCd2": "ALL",
    "codeNmisuCd_finder_stkisu4_3": "",
    "param1isuCd_finder_stkisu4_3": "KSQ",
    "majagntComCd": "",
    "tecComType": "TEC",
    "listTrack": "",
    "money": 1,
    "otherUnit": 1,
    "csvxls_isNo": False,
    "name": "fileDown",
    "url": "dbms/MDC/STAT/issue/MDCSTAT24401"
  }

headers = {'Referer' : 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0202',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

otp = requests.post(gen_otp_url, gen_otp, headers = headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_content = requests.post(down_url, {'code': otp}, headers = headers)

list_track = pd.read_csv(BytesIO(down_content.content), encoding = 'EUC-KR')

# 필요한 칼럼만 남기기
list_track = list_track[['회사명', '상장트랙', '주관사', '상장일', '공모가']].copy()

In [3]:
print('종목개수 : '+str(len(list_track))+'\n')

# HTML 변환 및 커스텀 태그로 감싸기
html_table = list_track.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
list_track

종목개수 : 151

<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>회사명</th>
      <th>상장트랙</th>
      <th>주관사</th>
      <th>상장일</th>
      <th>공모가</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>오름테라퓨틱</td>
      <td>기술성장기업(혁신기술기업)</td>
      <td>한국증권</td>
      <td>2025/02/14</td>
      <td>20000</td>
    </tr>
    <tr>
      <td>아이에스티이</td>
      <td>기술성장기업(혁신기술기업)</td>
      <td>KB증권</td>
      <td>2025/02/12</td>
      <td>11400</td>
    </tr>
    <tr>
      <td>아이지넷</td>
      <td>기술성장기업(사업모델기업)</td>
      <td>한국증권</td>
      <td>2025/02/04</td>
      <td>7000</td>
    </tr>
    <tr>
      <td>와이즈넛</td>
      <td>기술성장기업(혁신기술기업)</td>
      <td>삼성증권</td>
      <td>2025/01/24</td>
      <td>17000</td>
    </tr>
    <tr>
      <td>파인메딕스</td>
      <td>기술성장기업(혁신기술기업)</td>
      <td>한국증권</td>
      <td>2024/12/26</td>
      <td>10000</td>
    </tr>
    <tr>
      <td>쓰리에이로직스</td>
      <td>기술성장기업(혁신기술기업)</td>
      <td>신한증권,미래에셋증권<

,회사명,상장트랙,주관사,상장일,공모가
0,오름테라퓨틱,기술성장기업(혁신기술기업),한국증권,2025/02/14,20000
1,아이에스티이,기술성장기업(혁신기술기업),KB증권,2025/02/12,11400
2,아이지넷,기술성장기업(사업모델기업),한국증권,2025/02/04,7000
3,와이즈넛,기술성장기업(혁신기술기업),삼성증권,2025/01/24,17000
4,파인메딕스,기술성장기업(혁신기술기업),한국증권,2024/12/26,10000
...,...,...,...,...,...
146,제놀루션,기술성장기업(사업모델기업),신영증권,2020/07/24,14000
147,솔트룩스,기술성장기업(혁신기술기업),한국증권,2020/07/23,25000
148,소마젠,기술성장기업(혁신기술기업),신한증권,2020/07/13,11000
149,젠큐릭스,기술성장기업(혁신기술기업),미래에셋증권,2020/06/25,22700


### 종목코드 불러오기
그래도 한 회사에서 운영하는 같은 시스템이니, 종목코드를 vlookup거는 것은 어렵지는 않았습니다. 정보데이터시스템의 다른 화면 12005(전종목기본정보)에서 추가로 자료를 받아서 종목코드를 맵핑하는 것으로 해결은 가능합니다. 단지 조금 번거로울 뿐이죠.

In [4]:
#-----------------------------------------------------------
#정보데이터시스템에서 주권표준코드 불러오는 과정
#-----------------------------------------------------------

#화면번호 12005(전종목기본정보)
#거래소 홈페이지에서 불러오기
gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp = {
    'locale' : 'ko_KR',
    'mktId': 'ALL',
    'share': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT01901'
    }

headers = {'Referer' : 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

otp = requests.post(gen_otp_url, gen_otp, headers = headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_content = requests.post(down_url, {'code': otp}, headers = headers)


df_listed = pd.read_csv(BytesIO(down_content.content), encoding = 'EUC-KR')
df_listed['시장구분'] = df_listed['시장구분'].replace('KOSDAQ GLOBAL', 'KOSDAQ')
df_listed = df_listed.rename(columns=({'단축코드':'stock_code'}))

<br>회사명을 기준으로 종목코드를 맵핑하여 보겠습니다. 다행히 화면간***([20043] 공모가 대비 주가수익률, [12005]전종목기본정보)***{: style="color: #4682B4;"}에 회사명이 다른 경우는 없었습니다. (휴)
<br>맵핑한 결과는 아래와 같습니다.

In [5]:
# 상장트랙 데이터프레임에 종목코드를 매칭시키기
list_track_code = pd.merge(list_track, df_listed[['한글 종목약명','stock_code']], left_on = '회사명', right_on='한글 종목약명', how='left')

# 칼럼 순서 정비
list_track_code = list_track_code[['회사명', 'stock_code', '상장트랙', '주관사', '상장일', '공모가']]

In [6]:
# 종목코드가 매칭되지 않은 케이스는 없는지 확인
df_check = list_track_code[list_track_code['stock_code'].isna()]

if len(df_check) == 0:
  print('종목코드 전부 매칭됨')
else :
  # HTML 변환 및 커스텀 태그로 감싸기
  html_table = df_check.to_html(index=False, classes="dataframe", escape=False)

  # 테이블을 <div> 태그로 감싸기
  wrapped_html = f"""
  <div class="table-container">
    {html_table}
  </div>
  """
  print('종목코드가 매칭되지 않은 종목들 : \n')
  print(html_table)

종목코드 전부 매칭됨


In [7]:
# 데이터프레임 한번 확인해보기
print('종목개수 : '+str(len(list_track_code))+'\n')
# HTML 변환 및 커스텀 태그로 감싸기
html_table = list_track_code.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
list_track_code

종목개수 : 151

<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>회사명</th>
      <th>stock_code</th>
      <th>상장트랙</th>
      <th>주관사</th>
      <th>상장일</th>
      <th>공모가</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>오름테라퓨틱</td>
      <td>475830</td>
      <td>기술성장기업(혁신기술기업)</td>
      <td>한국증권</td>
      <td>2025/02/14</td>
      <td>20000</td>
    </tr>
    <tr>
      <td>아이에스티이</td>
      <td>212710</td>
      <td>기술성장기업(혁신기술기업)</td>
      <td>KB증권</td>
      <td>2025/02/12</td>
      <td>11400</td>
    </tr>
    <tr>
      <td>아이지넷</td>
      <td>462980</td>
      <td>기술성장기업(사업모델기업)</td>
      <td>한국증권</td>
      <td>2025/02/04</td>
      <td>7000</td>
    </tr>
    <tr>
      <td>와이즈넛</td>
      <td>096250</td>
      <td>기술성장기업(혁신기술기업)</td>
      <td>삼성증권</td>
      <td>2025/01/24</td>
      <td>17000</td>
    </tr>
    <tr>
      <td>파인메딕스</td>
      <td>387570</td>
      <td>기술성장기업(혁신기술기업)</td>
      <td>한국증권</td>
     

,회사명,stock_code,상장트랙,주관사,상장일,공모가
0,오름테라퓨틱,475830,기술성장기업(혁신기술기업),한국증권,2025/02/14,20000
1,아이에스티이,212710,기술성장기업(혁신기술기업),KB증권,2025/02/12,11400
2,아이지넷,462980,기술성장기업(사업모델기업),한국증권,2025/02/04,7000
3,와이즈넛,096250,기술성장기업(혁신기술기업),삼성증권,2025/01/24,17000
4,파인메딕스,387570,기술성장기업(혁신기술기업),한국증권,2024/12/26,10000
...,...,...,...,...,...,...
146,제놀루션,225220,기술성장기업(사업모델기업),신영증권,2020/07/24,14000
147,솔트룩스,304100,기술성장기업(혁신기술기업),한국증권,2020/07/23,25000
148,소마젠,950200,기술성장기업(혁신기술기업),신한증권,2020/07/13,11000
149,젠큐릭스,229000,기술성장기업(혁신기술기업),미래에셋증권,2020/06/25,22700


## KIND에서 기술성장기업 불러오기
한국거래소의 정보데이터시스템에는 없었지만, 한국거래소가 운영하는 **상장공시 배포시스템인 KIND**{: style="color: #4682B4;"}에는 혹시 있지 않을까? 라는 생각이 들어서 방문해보았습니다.

### KIND 속에 꽁꽁 숨어있는 기술성장기업 리스트
![kind1]({{site.url}}/assets/images/2025-03-01-listing/kind1.png)<br><br>
여기에도 IPO 관련 메뉴가 있습니다. 여기에 가보면 있겠군요!<br><br>
![kind2]({{site.url}}/assets/images/2025-03-01-listing/kind2.png)<br><br>
상장통계로 들어왔는데 유형별상장 통계만 있고 특례상장내용은 안보입니다. 실망하려던 찰나, **'기타상장통계' 메뉴**를 발견했습니다. 메인화면에서는 보이지 않고, ***'전체메뉴보기' - 'IPO현황' - '신규상장통계' - '상장통계' - '유형별상장통계'*** 경로로 접속하고 나면 보이는 메뉴입니다. <br><br>
![kind3]({{site.url}}/assets/images/2025-03-01-listing/kind3.png)<br><br>
연도별로 특례상장 기업 숫자가 나온 표가 있고, 숫자를 클릭하면 목록을 보여줍니다. **하지만 이번에도 종목코드는 없습니다.**{: style="color: #4682B4;"} <br><br>
![kind4]({{site.url}}/assets/images/2025-03-01-listing/kind4.png)<br><br>
그래도 정보데이터시스템은 5년치만 보여주는 반면, KIND에서는 좀 더 예전 기업까지 볼 수 있습니다. 현재 글을 작성중인 2025년 기준으로, 콤보박스에서 2018년까지 선택이 가능합니다. 그리고 2018년 화면으로 넘어가면 2014년까지 조회가 가능합니다. **어랏...? 기술특례 상장제도는 2005년에 처음 시작되었는데?** <br>
> **한국거래소 정보데이터시스템에서도, KIND에서도 기술성장기업 전체 리스트를 얻을 수는 없습니다.**{: style="color: #4682B4;"}
<br>

In [8]:
# URL 설정
url = 'https://kind.krx.co.kr/listinvstg/miscListTypeStatDetail.do'

# 여러 연도 데이터를 저장할 빈 데이터프레임 생성
df_special_kind = pd.DataFrame()

# 수집할 연도 리스트 설정
years = [i for i in range(2005,2026)]

for year in years:
    # 파라미터 설정
    params = {
        "method": "searchMiscListTypeStatDetailSub",
        "forward": "miscListTypeStatDetail_sub",
        "currentPageSize": 30,
        "pageIndex": 1,
        "listMonth": year,  # 연도 변경
        "marketType": 2,
        "listClssCd": 12
    }

    # POST 요청
    response = requests.post(url, params=params)
    response.encoding = 'utf-8'

    # HTML 파싱
    soup = BeautifulSoup(response.text, 'html.parser')

    # 테이블 찾기
    table = soup.find("table", {"class": "list type-99 mt5"})

    if table:
        # 테이블 헤더 추출
        headers = [th.text.strip() for th in table.find("thead").find_all("th")]

        # 테이블 데이터 추출
        data = []
        for row in table.find("tbody").find_all("tr"):
            cols = row.find_all("td")
            row_data = [col.text.strip() for col in cols]
            data.append(row_data)

        # 에러 대응
        try:
          # 데이터프레임 생성
          df_loop = pd.DataFrame(data, columns=headers)

          # 연도 칼럼 추가
          df_loop['연도'] = year

          # 전체 데이터프레임에 추가
          df_special_kind = pd.concat([df_special_kind, df_loop], ignore_index=True)

        except:
          print(f"{year}년 데이터를 찾을 수 없습니다.")
          pass
        # 서버 부하를 줄이기 위한 대기 시간
        sleep(1)
    else:
        print(f"{year}년 데이터를 찾을 수 없습니다.")

2005년 데이터를 찾을 수 없습니다.
2006년 데이터를 찾을 수 없습니다.
2007년 데이터를 찾을 수 없습니다.
2008년 데이터를 찾을 수 없습니다.
2009년 데이터를 찾을 수 없습니다.
2010년 데이터를 찾을 수 없습니다.
2011년 데이터를 찾을 수 없습니다.
2012년 데이터를 찾을 수 없습니다.
2013년 데이터를 찾을 수 없습니다.


In [9]:
# 데이터프레임 한번 확인해보기
print('종목개수 : '+str(len(df_special_kind))+'\n')
# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_special_kind.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_special_kind

종목개수 : 221

<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>회사명</th>
      <th>업종</th>
      <th>주요제품</th>
      <th>상장일(스팩합병일)</th>
      <th>결산기</th>
      <th>대표자명</th>
      <th>홈페이지</th>
      <th>지역</th>
      <th>연도</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>아스트</td>
      <td>항공기,우주선 및 부품 제조업</td>
      <td>항공기용 부품 제조 및 동체 조립</td>
      <td>2014-12-24</td>
      <td>12월</td>
      <td>김두일</td>
      <td>홈페이지 보기</td>
      <td>경상남도</td>
      <td>2014</td>
    </tr>
    <tr>
      <td>알테오젠</td>
      <td>자연과학 및 공학 연구개발업</td>
      <td>바이오시밀러 및 바이오베터</td>
      <td>2014-12-12</td>
      <td>12월</td>
      <td>박순재</td>
      <td>홈페이지 보기</td>
      <td>대전광역시</td>
      <td>2014</td>
    </tr>
    <tr>
      <td>덱스터</td>
      <td>영화, 비디오물, 방송프로그램 제작 및 배급업</td>
      <td>시각효과(Visual Effect, VFX)</td>
      <td>2015-12-22</td>
      <td>12월</td>
      <td>김욱, 강종익</td>
      <td>홈페이지 보기</td>
      <td>서울특별시</td>
      

,회사명,업종,주요제품,상장일(스팩합병일),결산기,대표자명,홈페이지,지역,연도
0,아스트,"항공기,우주선 및 부품 제조업",항공기용 부품 제조 및 동체 조립,2014-12-24,12월,김두일,홈페이지 보기,경상남도,2014
1,알테오젠,자연과학 및 공학 연구개발업,바이오시밀러 및 바이오베터,2014-12-12,12월,박순재,홈페이지 보기,대전광역시,2014
2,덱스터,"영화, 비디오물, 방송프로그램 제작 및 배급업","시각효과(Visual Effect, VFX)",2015-12-22,12월,"김욱, 강종익",홈페이지 보기,서울특별시,2015
3,HLB제약,자연과학 및 공학 연구개발업,"씨트리시메티딘정, 로자틴정",2015-12-21,12월31일,박재형,홈페이지 보기,경기도,2015
4,강스템바이오텍,자연과학 및 공학 연구개발업,줄기세포치료제,2015-12-21,12월,나종천,홈페이지 보기,서울특별시,2015
...,...,...,...,...,...,...,...,...,...
216,라메디텍,의료용 기기 제조업,초소형 레이저 의료기기 및 미용기기,2024-06-17,12월,최종석,홈페이지 보기,서울특별시,2024
217,오름테라퓨틱,기초 의약물질 제조업,항체약물접합체(ADC) 단백질 분해제(TPD) 연구 개발,2025-02-14,12월,이승주,홈페이지 보기,대전광역시,2025
218,아이에스티이,반도체 제조업,반도체 제조용 장비,2025-02-12,12월,조창현,홈페이지 보기,경기도,2025
219,아이지넷,기타 정보 서비스업,"보험 서비스 어플리케이션, 기업용 보험 솔루션",2025-02-04,12월,"김창균, 김지태",홈페이지 보기,서울특별시,2025


### 각 기업이 기술성장의 어느 트랙인지 확인하기
문제가 하나 더 있습니다. **KIND의 기타 상장통계화면에서는 혁신기술 트랙인지, 사업모델 트랙인지 구분하여 표시하고 있지 않습니다.**{: style="color: #4682B4;"} 정상적인 통계화면에서는 조회가능하지 않고... 우회적인 방법으로 트랙을 확인해야 합니다.<br><br>
![reportpage]({{site.url}}/assets/images/2025-03-01-listing/reportpage.png)<br><br>
KIND에는 ***'상장법인상세정보'-'기업분석보고서'-'사업모델평가보고서'***메뉴가 있습니다. 한국거래소의 코스닥 상장규정에서는 사업모델 트랙으로 상장한 특례기업의 상장을 주관한 **상장주관사에 성장성보고서(사업모델보고서) 제출을 의무화**{: style="color: #4682B4;"}하고 있습니다. 따라서 이 게시판에서 조회되는 회사는 사업모델 트랙으로 상장했음을 알 수 있습니다. <br>
이 게시판의 제출내역을 한번 봐보겠습니다.

In [10]:
# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# URL 및 파라미터 설정
url = 'https://kind.krx.co.kr/corpgeneral/growthReport.do'
params = {
    "method": "listingForeignCompanyList",
    "forward": "growthReportList",
    "currentPageSize": 50,
    "pageIndex": 1,
    "wrtrptType": 2,
    "contnId": 20241106000160,
    "wrtrptSeq": 0,
    "kosdaqBbsTpCd": 23,
    "gubun": "new",
    "marketType": "",
    "startDate": "1999-01-01",
    "endDate": today,
    "searchTextType": 1,
    "title": ""
}


# POST 요청
response = requests.post(url, params=params)
response.encoding = 'euc-kr'

# HTML 파싱
soup = BeautifulSoup(response.text, 'html.parser')

# 테이블 찾기
table = soup.find("table", {"class": "list type-00 tmt30"})

# 테이블 헤더 추출
headers = [th.text.strip() for th in table.find("thead").find_all("th")]

# 테이블 데이터 추출
data = []
for row in table.find("tbody").find_all("tr"):
    cols = row.find_all("td")
    row_data = [col.text.strip() for col in cols]
    data.append(row_data)

# 데이터프레임 생성
df_report = pd.DataFrame(data, columns=headers)

# 새로운 칼럼 추가
df_report['상장트랙'] = df_report['첨부'].apply(lambda x: '사업모델' if '사업모델' in x else ('사업모델' if '성장성' in x else '기타'))


In [11]:
# 데이터프레임 한번 확인해보기
print('개수 : '+str(len(df_report))+'\n')
# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_report.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_report

개수 : 21

<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>번호</th>
      <th>작성일</th>
      <th>회사명</th>
      <th>작성기관</th>
      <th>첨부</th>
      <th>상장트랙</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>21</td>
      <td>2024-11-06</td>
      <td>아이지넷</td>
      <td>한국투자증권</td>
      <td>상장주선인 추천에 의한 사업모델기업 평가보고서_아이지넷(게시용).pdf</td>
      <td>사업모델</td>
    </tr>
    <tr>
      <td>20</td>
      <td>2023-07-17</td>
      <td>와이랩</td>
      <td>한국투자증권</td>
      <td>상장주선인 추천에 의한 기술성장기업 성장성 보고서_와이랩_공시.pdf</td>
      <td>사업모델</td>
    </tr>
    <tr>
      <td>19</td>
      <td>2022-06-17</td>
      <td>선바이오</td>
      <td>하나증권</td>
      <td>기술성장기업 성장성 보고서_선바이오.pdf</td>
      <td>사업모델</td>
    </tr>
    <tr>
      <td>18</td>
      <td>2021-06-22</td>
      <td>원티드랩</td>
      <td>한국투자증권</td>
      <td>(주)원티드랩_성장성보고서.pdf</td>
      <td>사업모델</td>
    </tr>
    <tr>
      <td>17</td>
      <td>2021-04-02</td>
      <td>진시스템</td>
 

,번호,작성일,회사명,작성기관,첨부,상장트랙
0,21,2024-11-06,아이지넷,한국투자증권,상장주선인 추천에 의한 사업모델기업 평가보고서_아이지넷(게시용).pdf,사업모델
1,20,2023-07-17,와이랩,한국투자증권,상장주선인 추천에 의한 기술성장기업 성장성 보고서_와이랩_공시.pdf,사업모델
2,19,2022-06-17,선바이오,하나증권,기술성장기업 성장성 보고서_선바이오.pdf,사업모델
3,18,2021-06-22,원티드랩,한국투자증권,(주)원티드랩_성장성보고서.pdf,사업모델
4,17,2021-04-02,진시스템,삼성증권,[삼성증권]진시스템_상장주선인의 성장성보고서.pdf,사업모델
5,16,2021-03-23,삼영에스앤씨,미래에셋증권,삼영에스앤씨_상장주선인 추천에 의한 기술성장기업 성장성 보고서.pdf,사업모델
6,15,2021-01-15,레인보우로보틱스,미래에셋증권,레인보우로보틱스_상장주선인 추천에 의한 기술성장기업 성장성 보고서_vF.pdf,사업모델
7,14,2020-12-23,프레스티지바이오로직스,미래에셋증권,프레스티지바이오로직스_상장주선인 추천에 의한 기술성장기업 성장성 보고서.pdf,사업모델
8,13,2020-10-05,고바이오랩,삼성증권,고바이오랩_상장주선인의 성장성보고서.pdf,사업모델
9,12,2020-10-05,클리노믹스,대신증권,상장주선인 기술성장기업 성장성보고서_클리노믹스_20201005.pdf,사업모델


## KIND에서 코스닥 종목코드 불러오기
지금까지 KIND에서 2014년부터 현재까지 기술성장기업 목록과 상장트랙을 확인하였습니다. **그러나 여전히 종목코드가 없습니다.**{: style="color: #4682B4;"} <br>
아쉽게도 KIND에서 회사명과 종목코드를 일람표 형태로 보여주는 곳은 찾지 못했으나, 크롤링을 하는 과정에서 확인해보니 **html 소스에는 회사코드가 포함**{: style="color: #4682B4;"}된 경우가 대부분이었습니다. <br>
> KIND의 HTML 소스를 통해서 회사코드를 확인할 수 있습니다.
<br>

> 통상 종목코드는 회사코드 5자리 숫자 + 0입니다.
<br>

### 코스닥 상장법인 현황 불러오기
![kindlisting]({{site.url}}/assets/images/2025-03-01-listing/kindlisting.png)<br><br>
***'상장법인상세정보'-'상장법인목록'***을 들어가면 상장법인 목록이 있고, html소스에서 5자리 회사코드를 확인할 수 있습니다. 이를 활용하여 앞서 수집한 기술성장기업에 vlookup을 걸어보고, 혹시 vlookup이 안걸린 부분이 있는지 확인해보겠습니다.

In [12]:
url = 'https://kind.krx.co.kr/corpgeneral/corpList.do'
params = {
    "method": "searchCorpList",
    "pageIndex": 1,
    "currentPageSize": 2000,
    "comAbbrv": "",
    "beginIndex": "",
    "orderMode": 3,
    "orderStat": "D",
    "isurCd": "",
    "repIsuSrtCd": "",
    "searchCodeType": "",
    "marketType": "kosdaqMkt",
    "searchType": 13,
    "industry": "",
    "fiscalYearEnd": "all",
    "comAbbrvTmp": "",
    "location": "all"
}

# POST 요청
response = requests.post(url, params=params)
response.encoding = 'utf-8'

# HTML 파싱
soup = BeautifulSoup(response.text, 'html.parser')

# 테이블 찾기
table = soup.find("table", {"class": "list type-00 tmt30"})

# 칼럼명 직접 지정
headers = ['회사명', '업종', '주요제품', '상장일', '결산월', '대표자명', '지역', '회사코드']

# 데이터 추출
rows = []
for tr in table.find('tbody').find_all('tr'):
    tds = tr.find_all('td')

    # 필요한 데이터만 추출 (홈페이지 칼럼 제외)
    row_data = []
    for i, td in enumerate(tds):
        # 홈페이지 칼럼(인덱스 6) 건너뛰기
        if i != 6:
            row_data.append(td.text.strip())

    # 회사코드 추출
    company_link = tr.find('a', {'id': 'companysum'})
    if company_link:
        onclick_attr = company_link.get('onclick', '')
        company_code_match = re.search(r"companysummary_open\('([^']*)'\)", onclick_attr)
        if company_code_match:
            company_code = company_code_match.group(1)
            # 다섯 자리 숫자인지 확인하고 조건에 따라 처리
            if company_code.isdigit() and len(company_code) == 5:
                row_data.append(company_code + '0')
            else:
                row_data.append(company_code)
        else:
            row_data.append('')
    else:
        row_data.append('')

    rows.append(row_data)

# 데이터프레임 생성
df_kind = pd.DataFrame(rows, columns=headers)

In [13]:
# 데이터프레임 한번 확인해보기
print('개수 : '+str(len(df_kind))+'\n')
# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_kind.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_kind

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
      <td>컴퓨터 프로그래밍, 시스템 통합 및 관리업</td>
      <td>유해사이트차단서비스</td>
      <td>2005-06-10</td>
      <td>12월</td>
      <td>김태주..</td>
      <td>경기도</td>
      <td>075130</td>
    </tr>
    <tr>
      <td>이노와이어리스</td>
      <td>통신 및 방송 장비 제조업</td>
      <td>통신용 시험 및 계측장비</td>
      <td>2005-02-04</td>
      <td>12월</td>
      <td>곽영수</td>
      <td>경기도</td>
      <td>073490</td>
    </tr>
    <tr>
      <td>인베니아</td>
      <td>특수 목적용 기계 제조업</td>
      <td>Dry Etcher, 진공합착기, LCD Panel 검사장비, OLED용 진공합착기</td>
      <td>2005-02-04</td>
      <td>12월</td>
      <td>구동범</td>
      <td>경기도</td>
      <td>079950</td>
    </tr>
    <tr>
      <td>동양이엔피</td>
      <td>전동기, 발전기 및 전기 변환 · 공급 · 제어 장치 제조업</td>
      <td>휴대폰용,OA기기및통신장비용,디지털가전용전원공급장치</td>
      <td>2005-02-01</td>
      <td>12월</td>
      <td>김재수..</td>
      <td>경기도</td>
      <td>079960</td>
    </tr>
    <tr>
      <td>제주반도체</td>
      <td>반도체 제조업</td>
      <td>모바일용 LP DDR SDRAM, CRAM, SRAM</td>
   

,회사명,업종,주요제품,상장일,결산월,대표자명,지역,회사코드
0,에르코스,도시락 및 식사용 조리식품 제조업,"영유아식, 고령친화식품, 식물기반 대체식품",2025-02-28,12월,김슬기,대전광역시,435570
1,엘케이켐,기타 화학제품 제조업,"반도체 소재(CP, PCP, DIS) 등",2025-02-25,12월,이창엽,충청남도,489500
2,위너스,기타 전기장비 제조업,"배선기구(스위치, 콘센트, 멀티탭), 전기차 충전기(급속, 완속)",2025-02-24,12월,김창성,,479960
3,모티브링크,전자부품 제조업,"EV용 Main Transformer, EV용 Line Filter",2025-02-20,12월,김기한,경기도,463480
4,동국생명과학,의약품 제조업,"조영제 완제 및 원료, 의료기기 유통",2025-02-17,12월,박재원,서울특별시,303810
...,...,...,...,...,...,...,...,...
1783,하이록코리아,1차 철강 제조업,관이음쇠,1989-12-15,12월,문휴건..,부산광역시,013030
1784,SBI인베스트먼트,신탁업 및 집합투자업,벤처캐피탈,1989-09-06,12월,공동대..,서울특별시,019550
1785,엠벤처투자,신탁업 및 집합투자업,벤처캐피탈,1989-03-14,12월,심성보..,서울특별시,019590
1786,플루토스,신탁업 및 집합투자업,금융투자,1989-03-14,12월,엄원진,서울특별시,019570


### 회사명을 기준으로 종목코드를 vlookup 걸었는데... 누락이 있다!
**누락이 2종목 있습니다. 기술성장트랙으로 상장했으나 이미 상장폐지된 어스앤에어로스페이스, 유네코 입니다.**{: style="color: #4682B4;"} 앞서 크롤링했던 상장법인 목록이 '현재기준 상장된 회사'만 표출하다보니, 상장이 폐지된 회사에 대해선 크롤링이 안된 것이 이유였습니다.

In [14]:
# 특례상장목록 데이터프레임에 종목코드를 매칭시키기
df_special_kind_code = pd.merge(df_special_kind, df_kind[['회사명','회사코드']], on = '회사명', how='left')

In [15]:
# 종목코드가 매칭되지 않은 케이스는 없는지 확인
df_check = df_special_kind_code[df_special_kind_code['회사코드'].isna()]

if len(df_check) == 0:
  print('종목코드 전부 매칭됨')
else :
  # HTML 변환 및 커스텀 태그로 감싸기
  html_table = df_check.to_html(index=False, classes="dataframe", escape=False)

  # 테이블을 <div> 태그로 감싸기
  wrapped_html = f"""
  <div class="table-container">
    {html_table}
  </div>
  """
  print('종목코드가 매칭되지 않은 종목들 : \n')
  print(html_table)

df_check

종목코드가 매칭되지 않은 종목들 : 

<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>회사명</th>
      <th>업종</th>
      <th>주요제품</th>
      <th>상장일(스팩합병일)</th>
      <th>결산기</th>
      <th>대표자명</th>
      <th>홈페이지</th>
      <th>지역</th>
      <th>연도</th>
      <th>회사코드</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>어스앤에어로스페이스</td>
      <td>항공기,우주선 및 부품 제조업</td>
      <td>도어시스템</td>
      <td>2017-09-15</td>
      <td>12월31일</td>
      <td>조동우</td>
      <td>홈페이지 보기</td>
      <td>경상남도</td>
      <td>2017</td>
      <td>NaN</td>
    </tr>
    <tr>
      <td>유네코</td>
      <td>특수 목적용 기계 제조업</td>
      <td>SAP</td>
      <td>2018-03-15</td>
      <td>12월</td>
      <td>박동필, 김종원(각자대표)</td>
      <td>홈페이지 보기</td>
      <td>인천광역시</td>
      <td>2018</td>
      <td>NaN</td>
    </tr>
  </tbody>
</table>


,회사명,업종,주요제품,상장일(스팩합병일),결산기,대표자명,홈페이지,지역,연도,회사코드
26,어스앤에어로스페이스,"항공기,우주선 및 부품 제조업",도어시스템,2017-09-15,12월31일,조동우,홈페이지 보기,경상남도,2017,NaN
47,유네코,특수 목적용 기계 제조업,SAP,2018-03-15,12월,"박동필, 김종원(각자대표)",홈페이지 보기,인천광역시,2018,NaN


### 상장폐지법인 현황 불러와서 누락 메꾸기
***'투자유의사항'-'기타사항'-'상장폐지현황' 메뉴를 보면*** 상장폐지법인을 확인할 수 있고, html소스에서 회사코드도 따올 수 있습니다. 이 목록을 추가로 vlookup 걸어서 **기술성장기업 with 종목코드**{: style="color: #4682B4;"}를 만들어보겠습니다. <br><br>
![kinddel]({{site.url}}/assets/images/2025-03-01-listing/kinddel.png)<br><br>

In [16]:
url = 'https://kind.krx.co.kr/investwarn/delcompany.do'

params = {
    "method": "searchDelCompanySub",
    "currentPageSize": 5000,
    "pageIndex": 1,
    "orderMode": 2,
    "orderStat": "D",
    "tabType": 1,
    "searchMode": 1,
    "searchCodeType": "",
    "searchCorpName": "",
    "repIsuSrtCd": "",
    "forward": "delcompany_sub",
    "searchType": "",
    "marketType": 2,
    "searchCorpNameTmp": "",
    "fromDate": "1999-01-01",
    "toDate": today
}

# POST 요청
response = requests.post(url, params=params)
response.encoding = 'utf-8'

# HTML 파싱
soup = BeautifulSoup(response.text, 'html.parser')

# 테이블 찾기
table = soup.find("table", {"class": "list type-00 tmt30"})

# 테이블 헤더 추출
headers = ['번호', '회사명', '폐지일자', '폐지사유', '비고', '회사코드']


# 데이터 추출
rows = []
for tr in table.find('tbody').find_all('tr'):
    tds = tr.find_all('td')

    # 필요한 데이터만 추출 (홈페이지 칼럼 제외)
    row_data = []
    for i, td in enumerate(tds):
        # 홈페이지 칼럼(인덱스 6) 건너뛰기
        if i != 6:
            row_data.append(td.text.strip())

    # 회사코드 추출
    company_link = tr.find('a', {'id': 'companysum'})
    if company_link:
        onclick_attr = company_link.get('onclick', '')
        company_code_match = re.search(r"companysummary_open\('([^']*)'\)", onclick_attr)
        if company_code_match:
            company_code = company_code_match.group(1)
            # 다섯 자리 숫자인지 확인하고 조건에 따라 처리
            if company_code.isdigit() and len(company_code) == 5:
                row_data.append(company_code + '0')
            else:
                row_data.append(company_code)
        else:
            row_data.append('')
    else:
        row_data.append('')

    rows.append(row_data)

# 데이터프레임 생성
df_kind_del = pd.DataFrame(rows, columns=headers)

In [17]:
# 데이터프레임 한번 확인해보기
print('개수 : '+str(len(df_kind_del))+'\n')
# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_kind_del.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_kind_del

개수 : 980

<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>번호</th>
      <th>회사명</th>
      <th>폐지일자</th>
      <th>폐지사유</th>
      <th>비고</th>
      <th>회사코드</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>980</td>
      <td>키움제6호스팩</td>
      <td>2025-02-28</td>
      <td>피흡수합병(스팩소멸합병)</td>
      <td></td>
      <td>413600</td>
    </tr>
    <tr>
      <td>979</td>
      <td>LB루셈</td>
      <td>2025-02-21</td>
      <td>피흡수합병</td>
      <td></td>
      <td>376190</td>
    </tr>
    <tr>
      <td>978</td>
      <td>퀀타피아</td>
      <td>2025-02-19</td>
      <td>기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정</td>
      <td></td>
      <td>078940</td>
    </tr>
    <tr>
      <td>977</td>
      <td>이큐셀</td>
      <td>2025-02-14</td>
      <td>기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정</td>
      <td></td>
      <td>160600</td>
    </tr>
    <tr>
      <td>976</td>
      <td>애닉</td>
      <td>2025-02-14</td>
      <td>기

,번호,회사명,폐지일자,폐지사유,비고,회사코드
0,980,키움제6호스팩,2025-02-28,피흡수합병(스팩소멸합병),,413600
1,979,LB루셈,2025-02-21,피흡수합병,,376190
2,978,퀀타피아,2025-02-19,기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정,,078940
3,977,이큐셀,2025-02-14,기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정,,160600
4,976,애닉,2025-02-14,기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정,,299910
...,...,...,...,...,...,...
975,5,중앙금고,1999-03-11,주된영업의 양도,,025220
976,4,대진정밀화학,1999-02-12,등록법인의 취소신청,,033350
977,3,대성전기,1999-02-12,등록법인의 취소신청,,008580
978,2,전진산업,1999-01-15,등록법인의 취소신청,,000870


In [18]:
# 상장트랙 데이터프레임에 상장폐지회사의 종목코드를 매칭시키기
df_special_kind_code2 = pd.merge(df_special_kind_code, df_kind_del[['회사명','회사코드']], on = '회사명', how='left')

# 회사코드_x와 회사코드_y를 합쳐서 새로운 회사코드 칼럼 생성
df_special_kind_code2['회사코드'] = df_special_kind_code2['회사코드_x'].fillna(df_special_kind_code2['회사코드_y'])

# 기존 회사코드_x, 회사코드_y 칼럼 삭제 (선택사항)
df_special_kind_code2 = df_special_kind_code2.drop(['회사코드_x', '회사코드_y'], axis=1)

In [19]:
# 종목코드가 매칭되지 않은 케이스는 없는지 확인
df_check = df_special_kind_code2[df_special_kind_code2['회사코드'].isna()]

if len(df_check) == 0:
  print('종목코드 전부 매칭됨')
else :
  # HTML 변환 및 커스텀 태그로 감싸기
  html_table = df_check.to_html(index=False, classes="dataframe", escape=False)

  # 테이블을 <div> 태그로 감싸기
  wrapped_html = f"""
  <div class="table-container">
    {html_table}
  </div>
  """
  print('종목코드가 매칭되지 않은 종목들 : \n')
  print(html_table)

df_check

종목코드 전부 매칭됨


,회사명,업종,주요제품,상장일(스팩합병일),결산기,대표자명,홈페이지,지역,연도,회사코드


### 사업모델 보고서에 종목코드 붙이기
앞서 보았던 상장주관사가 제출하는 사업모델보고서(성장성보고서)에도 종목코드를 맵핑시켜 주겠습니다.
<br>

In [20]:
# 보고서 데이터프레임에 상장, 상장폐지회사의 종목코드를 매칭시키기
# 상장법인 현황
df_report_code = pd.merge(df_report, df_kind[['회사명','회사코드']], on = '회사명', how='left')

#상장폐지법인 현황
df_report_code2 = pd.merge(df_report_code, df_kind_del[['회사명','회사코드']], on = '회사명', how='left')

# 회사코드_x와 회사코드_y를 합쳐서 새로운 회사코드 칼럼 생성
df_report_code2['회사코드'] = df_report_code2['회사코드_x'].fillna(df_report_code2['회사코드_y'])

# 기존 회사코드_x, 회사코드_y 칼럼 삭제 (선택사항)
df_report_code2 = df_report_code2.drop(['회사코드_x', '회사코드_y'], axis=1)

In [21]:
# 종목코드가 매칭되지 않은 케이스는 없는지 확인
df_check = df_report_code2[df_report_code2['회사코드'].isna()]

if len(df_check) == 0:
  print('종목코드 전부 매칭됨')
else :
  # HTML 변환 및 커스텀 태그로 감싸기
  html_table = df_check.to_html(index=False, classes="dataframe", escape=False)

  # 테이블을 <div> 태그로 감싸기
  wrapped_html = f"""
  <div class="table-container">
    {html_table}
  </div>
  """
  print('종목코드가 매칭되지 않은 종목들 : \n')
  print(html_table)

df_check

종목코드 전부 매칭됨


,번호,작성일,회사명,작성기관,첨부,상장트랙,회사코드


### KIND에서 불러온 내용 전체 합치기
이제 KIND에서 불러온 아래의 정보들을 합쳐보겠습니다.<br>
* **기술성장기업 목록(종목코드 없음)**{: style="color: #4682B4;"}
* **주관사가 제출한 사업모델보고서(성장성보고서)(종목코드 없음)**{: style="color: #4682B4;"}
* **상장된 회사의 회사명과 회사코드**{: style="color: #4682B4;"}
* **상장폐지된 회사의 회사명과 회사코드**{: style="color: #4682B4;"}

<br>

In [22]:
# 보고서 데이터프레임에 상장, 상장폐지회사의 종목코드를 매칭시키기
# 상장법인 현황
df_kind_final = pd.merge(df_special_kind_code2, df_report_code2[['회사코드','상장트랙']], on = '회사코드', how='left')

# 상장트랙 중 비어있는 것은 '기술'로 기재하기
df_kind_final['상장트랙'] = df_kind_final['상장트랙'].fillna('혁신기술')

# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_kind_final.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_kind_final

<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>회사명</th>
      <th>업종</th>
      <th>주요제품</th>
      <th>상장일(스팩합병일)</th>
      <th>결산기</th>
      <th>대표자명</th>
      <th>홈페이지</th>
      <th>지역</th>
      <th>연도</th>
      <th>회사코드</th>
      <th>상장트랙</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>아스트</td>
      <td>항공기,우주선 및 부품 제조업</td>
      <td>항공기용 부품 제조 및 동체 조립</td>
      <td>2014-12-24</td>
      <td>12월</td>
      <td>김두일</td>
      <td>홈페이지 보기</td>
      <td>경상남도</td>
      <td>2014</td>
      <td>067390</td>
      <td>혁신기술</td>
    </tr>
    <tr>
      <td>알테오젠</td>
      <td>자연과학 및 공학 연구개발업</td>
      <td>바이오시밀러 및 바이오베터</td>
      <td>2014-12-12</td>
      <td>12월</td>
      <td>박순재</td>
      <td>홈페이지 보기</td>
      <td>대전광역시</td>
      <td>2014</td>
      <td>196170</td>
      <td>혁신기술</td>
    </tr>
    <tr>
      <td>덱스터</td>
      <td>영화, 비디오물, 방송프로그램 제작 및 배급업</td>
      <td>시각효과(Visual Effect, VFX)</td>
      

,회사명,업종,주요제품,상장일(스팩합병일),결산기,대표자명,홈페이지,지역,연도,회사코드,상장트랙
0,아스트,"항공기,우주선 및 부품 제조업",항공기용 부품 제조 및 동체 조립,2014-12-24,12월,김두일,홈페이지 보기,경상남도,2014,067390,혁신기술
1,알테오젠,자연과학 및 공학 연구개발업,바이오시밀러 및 바이오베터,2014-12-12,12월,박순재,홈페이지 보기,대전광역시,2014,196170,혁신기술
2,덱스터,"영화, 비디오물, 방송프로그램 제작 및 배급업","시각효과(Visual Effect, VFX)",2015-12-22,12월,"김욱, 강종익",홈페이지 보기,서울특별시,2015,206560,혁신기술
3,HLB제약,자연과학 및 공학 연구개발업,"씨트리시메티딘정, 로자틴정",2015-12-21,12월31일,박재형,홈페이지 보기,경기도,2015,047920,혁신기술
4,강스템바이오텍,자연과학 및 공학 연구개발업,줄기세포치료제,2015-12-21,12월,나종천,홈페이지 보기,서울특별시,2015,217730,혁신기술
...,...,...,...,...,...,...,...,...,...,...,...
216,라메디텍,의료용 기기 제조업,초소형 레이저 의료기기 및 미용기기,2024-06-17,12월,최종석,홈페이지 보기,서울특별시,2024,462510,혁신기술
217,오름테라퓨틱,기초 의약물질 제조업,항체약물접합체(ADC) 단백질 분해제(TPD) 연구 개발,2025-02-14,12월,이승주,홈페이지 보기,대전광역시,2025,475830,혁신기술
218,아이에스티이,반도체 제조업,반도체 제조용 장비,2025-02-12,12월,조창현,홈페이지 보기,경기도,2025,212710,혁신기술
219,아이지넷,기타 정보 서비스업,"보험 서비스 어플리케이션, 기업용 보험 솔루션",2025-02-04,12월,"김창균, 김지태",홈페이지 보기,서울특별시,2025,462980,사업모델


## 2014년 이전의 기술특례기업 목록은?

KIND에서도 2014년부터 기술성장기업의 목록을 제공합니다. 그이전에 상장한 기술성장기업 목록은 아쉽게도 한국거래소의 웹서비스를 통해서 구하진 못했습니다. <br>
여러경로로 검색하던 결과, 한국거래소와 인터뷰를 토대로 작성된 기사에 기술기업 목록이 기재된 것을 찾았습니다. <br>
[**거래소 기술기업상장부 인터뷰 기사(링크)**](https://www.newspim.com/news/view/20150310000431)
<br>
![listfromnews]({{site.url}}/assets/images/2025-03-01-listing/listfromnews.jpg)<br><br>

총 13개 종목이 확인되었으나, 여기에도 종목코드는 입력이 되어있지 않기 때문에 KIND에서 불러온 정보를 맵핑해보았습니다.
<br>


In [23]:
# 기사에서 확인한 2014년 이전 상장사 목록 입력하기
data_fromnews = {
    "회사명": [
        "바이로메드", "바이오니아", "크리스탈지노믹스", "이수앱지스", "제넥신", "진매트릭스",
        "인트론바이오", "나이벡", "디엔에이링크", '코렌텍', "레고켐바이오", '아미코젠', "인트로메딕"
    ],
    "상장일자": [
        "2005-12-29", "2005-12-29", "2006-01-06", "2009-02-03", "2009-09-15", "2009-11-06",
        "2011-01-26", "2011-07-13", "2011-12-26", "2013-03-05", "2013-05-10", "2013-09-12",
        "2013-12-19"
    ]
}

df_news = pd.DataFrame(data_fromnews)

### 종목코드 매칭해보기
세 개의 기업에 대해 종목코드가 맵핑되지 않았습니다. 셋 다 상장 이후에 회사명을 변경한 사례입니다..

In [24]:
# 특례상장목록 데이터프레임에 종목코드를 매칭시키기
df_news_code = pd.merge(df_news, df_kind[['회사명','회사코드']], on = '회사명', how='left')

# 데이터프레임 한번 확인해보기
print('개수 : '+str(len(df_news_code))+'\n')

# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_news_code.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_news_code

개수 : 13

<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>회사명</th>
      <th>상장일자</th>
      <th>회사코드</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>바이로메드</td>
      <td>2005-12-29</td>
      <td>NaN</td>
    </tr>
    <tr>
      <td>바이오니아</td>
      <td>2005-12-29</td>
      <td>064550</td>
    </tr>
    <tr>
      <td>크리스탈지노믹스</td>
      <td>2006-01-06</td>
      <td>NaN</td>
    </tr>
    <tr>
      <td>이수앱지스</td>
      <td>2009-02-03</td>
      <td>086890</td>
    </tr>
    <tr>
      <td>제넥신</td>
      <td>2009-09-15</td>
      <td>095700</td>
    </tr>
    <tr>
      <td>진매트릭스</td>
      <td>2009-11-06</td>
      <td>109820</td>
    </tr>
    <tr>
      <td>인트론바이오</td>
      <td>2011-01-26</td>
      <td>048530</td>
    </tr>
    <tr>
      <td>나이벡</td>
      <td>2011-07-13</td>
      <td>138610</td>
    </tr>
    <tr>
      <td>디엔에이링크</td>
      <td>2011-12-26</td>
      <td>127120</td>
    </tr>
    <tr>
      <td>코렌

,회사명,상장일자,회사코드
0,바이로메드,2005-12-29,NaN
1,바이오니아,2005-12-29,064550
2,크리스탈지노믹스,2006-01-06,NaN
3,이수앱지스,2009-02-03,086890
4,제넥신,2009-09-15,095700
5,진매트릭스,2009-11-06,109820
6,인트론바이오,2011-01-26,048530
7,나이벡,2011-07-13,138610
8,디엔에이링크,2011-12-26,127120
9,코렌텍,2013-03-05,104540


### 회사명이 바뀐 회사의 종목코드를 찾아서...
KIND에서는 변경전회사명을 입력하더라도 회사 검색이 가능합니다. 이를 활용하여 3개의 회사에 대해서 현재기준 회사명과 코드를 불러와 보았습니다.

In [25]:
# 종목코드가 없는 회사명 리스트 만들기
search_list = df_news_code[df_news_code['회사코드'].isna()]['회사명'].tolist()

In [26]:
# 결합할 데이터프레임 생성
df_search = pd.DataFrame()

for corpname in search_list :
  url = 'https://kind.krx.co.kr/common/corpList.do'
  params = {
      "method": "searchCorpList",
      "forward": "corpList",
      "pageIndex": 1,
      "beginIndex": "",
      "currentPageSize": 10,
      "delistFlag": "Y",
      "sub": "",
      "kwd": "",
      "searchCorp": "",
      "corpName": corpname,
      "corpNameTmp": "",
      "marketType": "all"
  }

  # POST 요청
  response = requests.post(url, params=params)
  response.encoding = 'utf-8'

  sleep(1)

  # HTML 파싱
  soup = BeautifulSoup(response.text, 'html.parser')

  # 테이블 찾기
  table = soup.find("table", {"class": "list type-99 mt20"})

  if table:

    # 테이블 헤더 추출
    headers = ['종목코드', '변경후회사명']

    # 테이블 데이터 추출
    data = []
    for row in table.find("tbody").find_all("tr"):
        cols = row.find_all("td")
        row_data = [col.text.strip() for col in cols]
        data.append(row_data)

        # 에러 대응
        try:
          # 데이터프레임 생성
          df_loop = pd.DataFrame(data, columns=headers)
          df_loop['회사명'] = corpname

          # 전체 데이터프레임에 추가
          df_search = pd.concat([df_search, df_loop], ignore_index=True)
        except:
          print(f"{corpname}을 찾을 수 없습니다.")
          pass

  else:
    print(f"{corpname}을 찾을 수 없습니다.")

In [27]:
# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_search.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_search

<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>종목코드</th>
      <th>변경후회사명</th>
      <th>회사명</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>084990</td>
      <td>헬릭스미스</td>
      <td>바이로메드</td>
    </tr>
    <tr>
      <td>083790</td>
      <td>CG인바이츠</td>
      <td>크리스탈지노믹스</td>
    </tr>
    <tr>
      <td>141080</td>
      <td>리가켐바이오</td>
      <td>레고켐바이오</td>
    </tr>
  </tbody>
</table>


,종목코드,변경후회사명,회사명
0,084990,헬릭스미스,바이로메드
1,083790,CG인바이츠,크리스탈지노믹스
2,141080,리가켐바이오,레고켐바이오


### 다시 종목코드 매칭해보기
회사명과 종목코드를 업데이트 하였으니 이제는 2014년 이전에 상장한 기술기업의 목록with 종목코드를 확보하였습니다!

In [28]:
# 특례상장목록 데이터프레임에 종목코드를 매칭시키기
df_news_code2 = pd.merge(df_news_code, df_search, on = '회사명', how='left')

# 회사코드를 합치기
df_news_code2['회사코드'] = df_news_code2['회사코드'].fillna(df_news_code2['종목코드'])

# 회사명을 합치기
df_news_code2['변경후회사명'] = df_news_code2['변경후회사명'].fillna(df_news_code2['회사명'])

# 기존 칼럼 삭제
df_news_code2 = df_news_code2.drop(['종목코드', '회사명'], axis=1)

# 변경후회사명 칼럼의 이름을 회사명으로 바꾸기
df_news_code2 = df_news_code2.rename(columns={'변경후회사명': '회사명'})

# 데이터프레임 한번 확인해보기
print('개수 : '+str(len(df_news_code2))+'\n')

# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_news_code2.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_news_code2

개수 : 13

<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>상장일자</th>
      <th>회사코드</th>
      <th>회사명</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>2005-12-29</td>
      <td>084990</td>
      <td>헬릭스미스</td>
    </tr>
    <tr>
      <td>2005-12-29</td>
      <td>064550</td>
      <td>바이오니아</td>
    </tr>
    <tr>
      <td>2006-01-06</td>
      <td>083790</td>
      <td>CG인바이츠</td>
    </tr>
    <tr>
      <td>2009-02-03</td>
      <td>086890</td>
      <td>이수앱지스</td>
    </tr>
    <tr>
      <td>2009-09-15</td>
      <td>095700</td>
      <td>제넥신</td>
    </tr>
    <tr>
      <td>2009-11-06</td>
      <td>109820</td>
      <td>진매트릭스</td>
    </tr>
    <tr>
      <td>2011-01-26</td>
      <td>048530</td>
      <td>인트론바이오</td>
    </tr>
    <tr>
      <td>2011-07-13</td>
      <td>138610</td>
      <td>나이벡</td>
    </tr>
    <tr>
      <td>2011-12-26</td>
      <td>127120</td>
      <td>디엔에이링크</td>
    </tr>
    <tr>
      <t

,상장일자,회사코드,회사명
0,2005-12-29,084990,헬릭스미스
1,2005-12-29,064550,바이오니아
2,2006-01-06,083790,CG인바이츠
3,2009-02-03,086890,이수앱지스
4,2009-09-15,095700,제넥신
5,2009-11-06,109820,진매트릭스
6,2011-01-26,048530,인트론바이오
7,2011-07-13,138610,나이벡
8,2011-12-26,127120,디엔에이링크
9,2013-03-05,104540,코렌텍


## 정보데이터시스템, KIND, 뉴스 종합하기!
일단 세 가지 소스에서 데이터를 준비해보았습니다. 아래의 도식대로 데이터를 처리하여 최종적으로 아래 데이터프레임을 구해보겠습니다.<br><br>
**기술성장기업**
* with 종목코드
* with 상장트랙

<br>

![datacomparison]({{site.url}}/assets/images/2025-03-01-listing/datacomparison.png)<br><br>

### 참고할 상장법인 정보를 가져오기
앞서 상장법인 목록을 통해서 상장사의 상장일 등의 정보를 가져온 적이 있었습니다. 근데 그 목록에는 상장폐지법인이 포함되어 있지 않았습니다. <br>
한편 상장폐지법인 목록에는 상장일 같은 정보는 없었습니다.<br>
그래서 차라리 '신규상장법인'목록을 표출해주는 화면에서 상장사 정보를 가져오기로 했습니다. 해당 화면에는 상장폐지된 법인도 표출이 되어있습니다.<br><br>
![kind6]({{site.url}}/assets/images/2025-03-01-listing/kind6.png)<br><br>
**...?**<br><br>
그렇네요.. 처음부터 '신규상장법인'목록을 통해서 가져왔으면 되는데 말이죠..ㅎㅎ 그랬으면 상장/상장폐지 화면을 돌아다니면서 가져올 필요가 없었습니다. 워낙 참고할 화면이 많다보니 이런 시행착오도 겪은 것이라 생각하겠습니다.

In [29]:
url = 'https://kind.krx.co.kr/listinvstg/listingcompany.do'
params = {
    "method": "searchListingTypeSub",
    "currentPageSize": 3000,
    "pageIndex": 1,
    "orderMode": 1,
    "orderStat": "D",
    "repIsuSrtCd": "",
    "isurCd": "",
    "forward": "listingtype_sub",
    "listTypeArrStr": "01|02|03|04|05|",
    "choicTypeArrStr": "",
    "searchCodeType": "",
    "searchCorpName": "",
    "secuGrpArrStr": "0|ST|FS|MF|SC|RT|DR|",
    "marketType": 2,
    "searchCorpNameTmp": "",
    "country": "",
    "industry": "",
    "repMajAgntDesignAdvserComp": "",
    "repMajAgntComp": "",
    "designAdvserComp": "",
    "secuGrpArr": ["0", "ST|FS", "MF|SC|RT", "DR"],
    "listTypeArr": ["01", "02", "03", "04", "05"],
    "fromDate": "2000-01-01",
    "toDate": today
}

# POST 요청
response = requests.post(url, params=params)
response.encoding = 'utf-8'

# HTML 파싱
soup = BeautifulSoup(response.text, 'html.parser')

# 테이블 찾기
table = soup.find("table", {"class": "list type-00 tmt30"})

if table:

  # 테이블 헤더 추출
  headers = ['회사명', '상장일', '상장유형', '증권구분','업종', '국적', '상장주선인', '회사코드']

  # 테이블 데이터 추출
  data = []
  for tr in table.find("tbody").find_all("tr"):
      cols = tr.find_all("td")
      row_data = [col.text.strip() for col in cols]

      # 회사코드 추출 - 새로운 패턴 적용
      onclick_attr = tr.get('onclick', '')
      company_code_match = re.search(r"fnDetailView\('([^']*)'", onclick_attr)

      if company_code_match:
          company_code = company_code_match.group(1)
          # 다섯 자리 숫자인지 확인하고 조건에 따라 처리
          if company_code.isdigit() and len(company_code) == 5:
              row_data.append(company_code + '0')
          else:
              row_data.append(company_code)
      else:
          row_data.append('')

      data.append(row_data)

  # 데이터프레임 생성 (반복문 밖에서 한 번만 실행)
  try:
      df_newlisting = pd.DataFrame(data, columns=headers)
      print(f"총 {len(df_newlisting)}개의 데이터를 찾았습니다.")
  except Exception as e:
      print(f"데이터프레임 생성 중 오류 발생: {e}")
      pass
else:
  print("찾을 수 없습니다.")

총 2237개의 데이터를 찾았습니다.


### 스팩합병일 경우는?
스팩합병 상장의 절차는 다음과 같습니다.
1. 먼저 껍데기 회사(스팩)을 상장시킵니다.
2. 상장된 스팩은 합병을 할 회사를 물색합니다.
3. 회사와 스팩이 합병하여 진짜 상장이 이루어집니다.

<br>
위에서 가져온 신규상장법인 현황의 경우, **스팩존속 합병법인에 대해서는 스팩상장일이 기재**{: style="color: #4682B4;"}
되어 있습니다. 우리가 궁금한건 '진짜 상장일'이니, 스팩합병상장일을 추가로 구해와야 합니다. KIND에는 아래와 같은 화면에서 이를 확인할 수 있습니다.
<br><br>

![kind7]({{site.url}}/assets/images/2025-03-01-listing/kind7.png)<br><br>


In [30]:
url = 'https://kind.krx.co.kr/listinvstg/mergeListingCompany.do'
params = {
    "method": "searchMergeListingCompSub",
    "currentPageSize": 2000,
    "pageIndex": 1,
    "orderMode": 1,
    "orderStat": "D",
    "repIsuSrtCd": "",
    "isurCd": "",
    "forward": "mergeListingCompany_sub",
    "listTypeArrStr": "06|07|",
    "choicTypeArrStr": "",
    "searchCodeType": "",
    "searchCorpName": "",
    "secuGrpArrStr": "0|ST|FS|MF|SC|RT|DR|",
    "marketType": 2,
    "searchCorpNameTmp": "",
    "country": "",
    "industry": "",
    "repMajAgntDesignAdvserComp": "",
    "repMajAgntComp": "",
    "designAdvserComp": "",
    "secuGrpArr": ["0", "ST|FS", "MF|SC|RT", "DR"],
    "listTypeArr": ["06", "07"],
    "fromDate": "2000-01-01",
    "toDate": today
}

# POST 요청
response = requests.post(url, params=params)
response.encoding = 'utf-8'

# HTML 파싱
soup = BeautifulSoup(response.text, 'html.parser')

# 테이블 찾기
table = soup.find("table", {"class": "list type-00 tmt30"})

if table:

  # 테이블 헤더 추출
  headers = ['회사명', '합병상장일', '상장유형', '증권구분','업종', '국적', '상장주선인', '회사코드']

  # 테이블 데이터 추출
  data = []
  for tr in table.find("tbody").find_all("tr"):
      cols = tr.find_all("td")
      row_data = [col.text.strip() for col in cols]

      # 회사코드 추출 - 새로운 패턴 적용
      onclick_attr = tr.get('onclick', '')
      company_code_match = re.search(r"fnDetailView\('([^']*)'", onclick_attr)

      if company_code_match:
          company_code = company_code_match.group(1)
          # 다섯 자리 숫자인지 확인하고 조건에 따라 처리
          if company_code.isdigit() and len(company_code) == 5:
              row_data.append(company_code + '0')
          else:
              row_data.append(company_code)
      else:
          row_data.append('')

      data.append(row_data)

  # 데이터프레임 생성 (반복문 밖에서 한 번만 실행)
  try:
      df_spac = pd.DataFrame(data, columns=headers)
      df_spac = df_spac.rename(columns={'상장유형': '합병상장유형'})
      print(f"총 {len(df_spac)}개의 데이터를 찾았습니다.")
  except Exception as e:
      print(f"데이터프레임 생성 중 오류 발생: {e}")
      pass
else:
  print("찾을 수 없습니다.")

총 165개의 데이터를 찾았습니다.


In [31]:
# 기존 신규상장 목록에 스팩상장 관련 내용 붙이기
df_newlisting2 = pd.merge(df_newlisting, df_spac[['회사코드','합병상장일','합병상장유형']], on = '회사코드', how='left')

# 잘 붙었는지 확인해보기(상위 5개행)
df_show = df_newlisting2[~df_newlisting2['합병상장일'].isna()].head()

# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_show.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_show

<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>회사명</th>
      <th>상장일</th>
      <th>상장유형</th>
      <th>증권구분</th>
      <th>업종</th>
      <th>국적</th>
      <th>상장주선인</th>
      <th>회사코드</th>
      <th>합병상장일</th>
      <th>합병상장유형</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>IBKS제18호스팩</td>
      <td>2022-07-22</td>
      <td>신규상장</td>
      <td>주권</td>
      <td>금융 지원 서비스업</td>
      <td>대한민국</td>
      <td>IBK투자증권(주)</td>
      <td>424760</td>
      <td>2023-04-28</td>
      <td>SPAC 존속합병</td>
    </tr>
    <tr>
      <td>유진스팩8호</td>
      <td>2022-03-31</td>
      <td>신규상장</td>
      <td>주권</td>
      <td>금융 지원 서비스업</td>
      <td>대한민국</td>
      <td>유진투자증권(주)</td>
      <td>413630</td>
      <td>2024-06-27</td>
      <td>SPAC 존속합병</td>
    </tr>
    <tr>
      <td>에스케이증권7호스팩</td>
      <td>2022-03-07</td>
      <td>신규상장</td>
      <td>주권</td>
      <td>금융 지원 서비스업</td>
      <td>대한민국</td>
      <td>SK증권(주)</td>
      <

,회사명,상장일,상장유형,증권구분,업종,국적,상장주선인,회사코드,합병상장일,합병상장유형
300,IBKS제18호스팩,2022-07-22,신규상장,주권,금융 지원 서비스업,대한민국,IBK투자증권(주),424760,2023-04-28,SPAC 존속합병
327,유진스팩8호,2022-03-31,신규상장,주권,금융 지원 서비스업,대한민국,유진투자증권(주),413630,2024-06-27,SPAC 존속합병
332,에스케이증권7호스팩,2022-03-07,신규상장,주권,금융 지원 서비스업,대한민국,SK증권(주),408920,2023-03-03,SPAC 존속합병
376,엔에이치스팩21호,2021-10-15,신규상장,주권,금융 지원 서비스업,대한민국,엔에이치투자증권주식회사,391710,2022-07-29,SPAC 존속합병
391,IBKS제16호스팩,2021-09-03,신규상장,주권,금융 지원 서비스업,대한민국,IBK투자증권(주),388790,2023-02-14,SPAC 존속합병


### KIND와 뉴스를 먼저 종합하기
일단 KIND와 뉴스기사를 통해서 제도 도입이래 기술성장기업 목록을 확보합니다.

In [32]:
# 2014년 전에는 사업모델 트랙이 없었으므로 기술로 구분
df_news_code2['상장트랙'] = '혁신기술'

# KIND와 뉴스 데이터프레임 쌓기
df_special_all = pd.concat([df_news_code2[['회사명','회사코드', '상장트랙']],
                            df_kind_final[['회사명','회사코드', '상장트랙']]],
                           axis = 0)

In [33]:
# 합치기
df_special_all2 = pd.merge(df_special_all, df_newlisting2, on = '회사코드', how='left')

# 회사명x는 회사명으로, 회사명y는 상장당시회사명으로 변경하기
df_special_all2 = df_special_all2.rename(columns={'회사명_x': '회사명', '회사명_y': '상장당시회사명'})

# 업종 칼럼 지우기
df_special_all2 = df_special_all2.drop(['업종'], axis=1)

# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_special_all2.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_special_all2

<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>회사명</th>
      <th>회사코드</th>
      <th>상장트랙</th>
      <th>상장당시회사명</th>
      <th>상장일</th>
      <th>상장유형</th>
      <th>증권구분</th>
      <th>국적</th>
      <th>상장주선인</th>
      <th>합병상장일</th>
      <th>합병상장유형</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>헬릭스미스</td>
      <td>084990</td>
      <td>혁신기술</td>
      <td>바이로메드</td>
      <td>2005-12-29</td>
      <td>신규상장</td>
      <td>주권</td>
      <td>대한민국</td>
      <td>KB증권(주)</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <td>바이오니아</td>
      <td>064550</td>
      <td>혁신기술</td>
      <td>바이오니아</td>
      <td>2005-12-29</td>
      <td>신규상장</td>
      <td>주권</td>
      <td>대한민국</td>
      <td>미래에셋증권 주식회사</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <td>CG인바이츠</td>
      <td>083790</td>
      <td>혁신기술</td>
      <td>크리스탈지노믹스</td>
      <td>2006-01-06</td>
      <td>신규상장</td>
      <td>

,회사명,회사코드,상장트랙,상장당시회사명,상장일,상장유형,증권구분,국적,상장주선인,합병상장일,합병상장유형
0,헬릭스미스,084990,혁신기술,바이로메드,2005-12-29,신규상장,주권,대한민국,KB증권(주),NaN,NaN
1,바이오니아,064550,혁신기술,바이오니아,2005-12-29,신규상장,주권,대한민국,미래에셋증권 주식회사,NaN,NaN
2,CG인바이츠,083790,혁신기술,크리스탈지노믹스,2006-01-06,신규상장,주권,대한민국,미래에셋증권 주식회사,NaN,NaN
3,이수앱지스,086890,혁신기술,이수앱지스,2009-02-03,신규상장,주권,대한민국,삼성증권(주),NaN,NaN
4,제넥신,095700,혁신기술,제넥신,2009-09-15,신규상장,주권,대한민국,"교보증권(주),미래에셋증권 주식회사",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
229,라메디텍,462510,혁신기술,라메디텍,2024-06-17,신규상장,주권,대한민국,대신증권(주),NaN,NaN
230,오름테라퓨틱,475830,혁신기술,오름테라퓨틱,2025-02-14,신규상장,주권,대한민국,한국투자증권(주),NaN,NaN
231,아이에스티이,212710,혁신기술,아이에스티이,2025-02-12,신규상장,주권,대한민국,KB증권(주),NaN,NaN
232,아이지넷,462980,사업모델,아이지넷,2025-02-04,신규상장,주권,대한민국,한국투자증권(주),NaN,NaN


### 외구주권,DR 종목코드 받아오기
정보데이터시스템과 비교하기 위해선 종목코드는 반드시 필요한데 말이죠...,<br>
> 외국회사와 KDR(주식예탁증권)의 경우, 회사코드에 0을 붙여서 보통주 종목코드가 되는 구조가 아닐 수 있습니다.

<br>
따라서 마지막으로 KIND에서 외국회사, KDR의 종목코드까지만 구하고 다음단계로 진행해보겠습니다. 우선 외국회사와 KDR의 목록을 KIND에서 확인해보겠습니다.

In [34]:
# 날짜 형태가 YYYYMMDD이므로 형태를 변경
today2 = today[:4]+today[5:7]+today[8:]

# DR과 외국주권을 대상으로 하는 리스트
target = ['DR', 'FS']

# 취합할 데이터프레임
df_drfs = pd.DataFrame()

for target in target :
  url = 'https://kind.krx.co.kr/corpgeneral/listedissuestatusdetail.do'
  params = {
      "method": "searchListedIssueStatDetailSub",
      "forward": "listedissuestatdetail_sub",
      "currentPageSize": 500,
      "pageIndex": 1,
      "selDate": today2,
      "mktId": "KSQ",
      "secugrpId": target,
      "detailType": 1
  }

  # POST 요청
  response = requests.post(url, params=params)
  response.encoding = 'utf-8'

  # HTML 파싱
  soup = BeautifulSoup(response.text, 'html.parser')

  # 테이블 찾기
  table = soup.find("table", {"class": "list type-99 mt5"})

  if table:

    # 테이블 헤더 추출
    headers = ['구분', '회사명', '종목코드', '상장일','상장주식']

    # 테이블 데이터 추출
    data = []
    for tr in table.find("tbody").find_all("tr"):
        cols = tr.find_all("td")
        row_data = [col.text.strip() for col in cols]

        data.append(row_data)

    # 데이터프레임 생성
    try:
        df_loop = pd.DataFrame(data, columns=headers)
        df_loop['구분'] = target
        df_drfs = pd.concat([df_drfs, df_loop], ignore_index=True)
    except Exception as e:
        print(f"데이터프레임 생성 중 오류 발생: {e}")
        pass
  else:
    print("찾을 수 없습니다.")

In [35]:
# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_drfs.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_drfs

<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>구분</th>
      <th>회사명</th>
      <th>종목코드</th>
      <th>상장일</th>
      <th>상장주식</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>DR</td>
      <td>네오이뮨텍</td>
      <td>950220</td>
      <td>2021-03-16</td>
      <td>98,867</td>
    </tr>
    <tr>
      <td>DR</td>
      <td>고스트스튜디오</td>
      <td>950190</td>
      <td>2020-08-18</td>
      <td>13,580</td>
    </tr>
    <tr>
      <td>DR</td>
      <td>소마젠</td>
      <td>950200</td>
      <td>2020-07-13</td>
      <td>19,236</td>
    </tr>
    <tr>
      <td>DR</td>
      <td>JTC</td>
      <td>950170</td>
      <td>2018-04-06</td>
      <td>51,746</td>
    </tr>
    <tr>
      <td>DR</td>
      <td>코오롱티슈진</td>
      <td>950160</td>
      <td>2017-11-06</td>
      <td>81,524</td>
    </tr>
    <tr>
      <td>DR</td>
      <td>잉글우드랩</td>
      <td>950140</td>
      <td>2016-10-14</td>
      <td>19,868</td>
    </tr>
    <tr>
    

,구분,회사명,종목코드,상장일,상장주식
0,DR,네오이뮨텍,950220,2021-03-16,"98,867"
1,DR,고스트스튜디오,950190,2020-08-18,"13,580"
2,DR,소마젠,950200,2020-07-13,"19,236"
3,DR,JTC,950170,2018-04-06,"51,746"
4,DR,코오롱티슈진,950160,2017-11-06,"81,524"
5,DR,잉글우드랩,950140,2016-10-14,"19,868"
6,DR,엑세스바이오,950130,2013-05-30,"37,728"
7,DR,SBI핀테크솔루션즈,950110,2012-12-17,"24,053"
8,FS,윙입푸드,900340,2018-11-30,"50,331"
9,FS,컬러레이,900310,2017-08-10,"64,042"


<br>총 19개의 종목이 있습니다. 이제 우리가 앞서 구해보았던 기술성장기업 중에 종목코드가 숫자 6자리가 아닌 것을 찾아서 숫자 6자리로 교체해주도록 하겠습니다.<br>
**네오이뮨텍(USA14), 소마젠(USA15)가 여기에 해당하여 수정해주었습니다.**{: style="color: #4682B4;"}


In [36]:
# 회사코드가 숫자 6개가 아닌 목록 확인하기
df_check = df_special_all2[~((df_special_all2['회사코드'].str.isdigit()) & (df_special_all2['회사코드'].str.len() == 6))]
# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_check.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_check

<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>회사명</th>
      <th>회사코드</th>
      <th>상장트랙</th>
      <th>상장당시회사명</th>
      <th>상장일</th>
      <th>상장유형</th>
      <th>증권구분</th>
      <th>국적</th>
      <th>상장주선인</th>
      <th>합병상장일</th>
      <th>합병상장유형</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>소마젠</td>
      <td>USA15</td>
      <td>혁신기술</td>
      <td>소마젠</td>
      <td>2020-07-13</td>
      <td>신규상장</td>
      <td>주식예탁증권</td>
      <td>미국</td>
      <td>신한투자증권 주식회사</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <td>네오이뮨텍</td>
      <td>USA14</td>
      <td>혁신기술</td>
      <td>네오이뮨텍</td>
      <td>2021-03-16</td>
      <td>신규상장</td>
      <td>주식예탁증권</td>
      <td>미국</td>
      <td>미래에셋증권 주식회사,하나증권주식회사</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
  </tbody>
</table>


,회사명,회사코드,상장트랙,상장당시회사명,상장일,상장유형,증권구분,국적,상장주선인,합병상장일,합병상장유형
106,소마젠,USA15,혁신기술,소마젠,2020-07-13,신규상장,주식예탁증권,미국,신한투자증권 주식회사,NaN,NaN
134,네오이뮨텍,USA14,혁신기술,네오이뮨텍,2021-03-16,신규상장,주식예탁증권,미국,"미래에셋증권 주식회사,하나증권주식회사",NaN,NaN


In [37]:
# 확인된 회사명을 리스트로 만들기
change_list = df_check['회사명'].tolist()

# change_list의 각 회사명에 대해 반복
for company in change_list:
    # df_drfs에서 해당 회사명에 맞는 종목코드 찾기
    if company in df_drfs['회사명'].values:
        # 해당 회사의 종목코드 가져오기
        code = df_drfs.loc[df_drfs['회사명'] == company, '종목코드'].values[0]

        # df_special_all2에서 해당 회사명에 맞는 행 찾아 회사코드 업데이트
        df_special_all2.loc[df_special_all2['회사명'] == company, '회사코드'] = code

        print(f"{company}의 회사코드가 {code}로 변경되었습니다.")
    else:
        print(f"{company}는 df_drfs에 존재하지 않습니다.")


소마젠의 회사코드가 950200로 변경되었습니다.
네오이뮨텍의 회사코드가 950220로 변경되었습니다.


### 정보데이터시스템과도 합쳐서 정합성 확인하기
정보데이터시스템은 5년치만 있지만 상장트랙(기술/사업모델)을 구분하는 정합성이 높습니다. <br><br>
![listingtrack]({{site.url}}/assets/images/2025-03-01-listing/listingtrack.png)<br><br>

따라서 5년치 데이터에 대해서는 매칭시켜서 혹시 불일치하는 것은 없는지 확인해보겠습니다. 결과적으로 불일치하는 것은 없었습니다!


In [38]:
# 데이터프레임을 합치기
df_special_all3 = pd.merge(df_special_all2, list_track_code[['stock_code','상장트랙','주관사']],
                           left_on = '회사코드', right_on = 'stock_code', how='left')

# vlookup이 걸린 행만 확인해보기
df_precise = df_special_all3[~df_special_all3['stock_code'].isna()]
# 정보데이터시스템의 상장유형을 확인하기
print(df_precise['상장트랙_y'].unique())

['기술성장기업(혁신기술기업)' '기술성장기업(사업모델기업)']


In [40]:
# 두 종류가 있으므로, 대사를 위해 값을 통일
df_precise.loc[:, '상장트랙_z'] = np.where(df_precise['상장트랙_y'] == '기술성장기업(혁신기술기업)', '혁신기술',
                                      np.where(df_precise['상장트랙_y'] == '기술성장기업(사업모델기업)', '사업모델', '기타'))

# 값이 불일치 하는 경우 표출
df_check = df_precise[df_precise['상장트랙_x'] != df_precise['상장트랙_z']]
# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_check.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_check

<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>회사명</th>
      <th>회사코드</th>
      <th>상장트랙_x</th>
      <th>상장당시회사명</th>
      <th>상장일</th>
      <th>상장유형</th>
      <th>증권구분</th>
      <th>국적</th>
      <th>상장주선인</th>
      <th>합병상장일</th>
      <th>합병상장유형</th>
      <th>stock_code</th>
      <th>상장트랙_y</th>
      <th>주관사</th>
      <th>상장트랙_z</th>
    </tr>
  </thead>
  <tbody>
  </tbody>
</table>


,회사명,회사코드,상장트랙_x,상장당시회사명,상장일,상장유형,증권구분,국적,상장주선인,합병상장일,합병상장유형,stock_code,상장트랙_y,주관사,상장트랙_z


## 2025년 2월말 기준 기술성장기업 목록 완성
마침내 완성했습니다. KIND, 정보데이터시스템, 기사를 통해 확인된 2025년 2월말 기준 역대 기술성장기업은 234사였습니다.

In [41]:
# 정합성에 문제가 없었으므로 기존의 df_special_all2가 최종
# 회사 개수 표출
print('기술성장기업 개수'+ str(len(df_special_all2)))
# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_special_all2.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_special_all2

기술성장기업 개수234
<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>회사명</th>
      <th>회사코드</th>
      <th>상장트랙</th>
      <th>상장당시회사명</th>
      <th>상장일</th>
      <th>상장유형</th>
      <th>증권구분</th>
      <th>국적</th>
      <th>상장주선인</th>
      <th>합병상장일</th>
      <th>합병상장유형</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>헬릭스미스</td>
      <td>084990</td>
      <td>혁신기술</td>
      <td>바이로메드</td>
      <td>2005-12-29</td>
      <td>신규상장</td>
      <td>주권</td>
      <td>대한민국</td>
      <td>KB증권(주)</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <td>바이오니아</td>
      <td>064550</td>
      <td>혁신기술</td>
      <td>바이오니아</td>
      <td>2005-12-29</td>
      <td>신규상장</td>
      <td>주권</td>
      <td>대한민국</td>
      <td>미래에셋증권 주식회사</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <td>CG인바이츠</td>
      <td>083790</td>
      <td>혁신기술</td>
      <td>크리스탈지노믹스</td>
      <td>2006-01-06</td>
      <td>신규상장</t

,회사명,회사코드,상장트랙,상장당시회사명,상장일,상장유형,증권구분,국적,상장주선인,합병상장일,합병상장유형
0,헬릭스미스,084990,혁신기술,바이로메드,2005-12-29,신규상장,주권,대한민국,KB증권(주),NaN,NaN
1,바이오니아,064550,혁신기술,바이오니아,2005-12-29,신규상장,주권,대한민국,미래에셋증권 주식회사,NaN,NaN
2,CG인바이츠,083790,혁신기술,크리스탈지노믹스,2006-01-06,신규상장,주권,대한민국,미래에셋증권 주식회사,NaN,NaN
3,이수앱지스,086890,혁신기술,이수앱지스,2009-02-03,신규상장,주권,대한민국,삼성증권(주),NaN,NaN
4,제넥신,095700,혁신기술,제넥신,2009-09-15,신규상장,주권,대한민국,"교보증권(주),미래에셋증권 주식회사",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
229,라메디텍,462510,혁신기술,라메디텍,2024-06-17,신규상장,주권,대한민국,대신증권(주),NaN,NaN
230,오름테라퓨틱,475830,혁신기술,오름테라퓨틱,2025-02-14,신규상장,주권,대한민국,한국투자증권(주),NaN,NaN
231,아이에스티이,212710,혁신기술,아이에스티이,2025-02-12,신규상장,주권,대한민국,KB증권(주),NaN,NaN
232,아이지넷,462980,사업모델,아이지넷,2025-02-04,신규상장,주권,대한민국,한국투자증권(주),NaN,NaN


<br><br>
간단할 줄 알알았던 이 과제는 이토록 먼길을 돌아오게 되었습니다. 2014년 전에 상장한 기술기업 목록이 한국거래소에서는 제공되지 않아서 인터넷을 검색하던 중, 기술성장 트랙 상장에 대해 컨설팅을 제공하는 업체의 사이트를 우연히 방문하게 되었습니다. 여기의 Top조회 게시물은 기술성장기업의 목록이었습니다. 투자의 목적이든 상장을 준비하는 회사의 입장이던 시장에서 충분히 궁금해할 만한 수요있는 정보라는 생각이 들었습니다. 앞으로도 시장을 더 알아가는 컨텐츠를 만들어보겠습니다. <br>

![rise]({{site.url}}/assets/images/2025-03-01-listing/rise.png)<br><br>